In [1]:
import pyspark
from pyspark.sql import SparkSession
import os

os.environ['AWS_REGION'] = "eu-west-1"
conf = (
    pyspark.SparkConf()
        .setAppName('iceberg-data-explorer')
        .set("spark.sql.catalog.rest", "org.apache.iceberg.spark.SparkCatalog")
        .set("spark.sql.catalog.rest.catalog-impl", "org.apache.iceberg.rest.RESTCatalog")
        .set("spark.sql.catalog.rest.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
        .set("spark.sql.catalog.rest.uri", "http://iceberg-catalog:8181/")
        .set("spark.sql.catalog.rest.s3.endpoint", "http://minio:9000")
        .set("spark.sql.catalog.rest.s3.path-style-access", "true")
        .set("spark.sql.defaultCatalog", "rest")
)

## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

spark.sql("SELECT * FROM iceberg.application_db_public_products").show()

Spark Running
+---+--------------+------------+----+--------+--------------+-----------+---------+--------------------+
| id|          name|       price|__op| __table|__source_ts_ms|       __db|__deleted|         __source_ts|
+---+--------------+------------+----+--------+--------------+-----------+---------+--------------------+
|  1|          Book|{2, [02 6E]}|   d|products| 1695380454902|application|     true|2023-09-22 11:00:...|
|  3|Remote Control|{2, [04 13]}|   d|products| 1695295333326|application|     true|2023-09-21 11:22:...|
|  5|         Shoes|{2, [00 8F]}|   u|products| 1695380439311|application|    false|2023-09-22 11:00:...|
|  2|         Chair|{2, [09 CD]}|   r|products| 1695288693878|application|    false|2023-09-21 09:31:...|
|  3|Remote Control|{2, [04 13]}|   c|products| 1695380350829|application|    false|2023-09-22 10:59:...|
+---+--------------+------------+----+--------+--------------+-----------+---------+--------------------+

